# Wine Quality — Preprocessing Notebook

Step-by-step preprocessing for `/mnt/data/wine-quality.csv` with short explanations. Run cells sequentially.

## 1) Imports

Import libraries used for data handling, preprocessing and saving outputs.

In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import joblib

print('Libraries imported')

## 2) Load dataset

Read the CSV file into a DataFrame and show shape & first rows.

In [ ]:
DATA_PATH = '/mnt/data/wine-quality.csv'

assert os.path.exists(DATA_PATH), f"File not found: {DATA_PATH}"

df = pd.read_csv(DATA_PATH)
print('Shape:', df.shape)
df.head(8)

## 3) Info & basic statistics

Check column types and summary statistics to understand the data.

In [ ]:
# DataFrame info
print('\nInfo:')
df.info()

# Describe numeric columns
print('\nDescribe:')
df.describe().T

## 4) Missing values

Identify any missing values per column.

In [ ]:
print('Missing values per column:')
df.isna().sum()

## 5) Duplicate rows

Check and optionally remove exact duplicate rows.

In [ ]:
dup_count = df.duplicated().sum()
print('Duplicate rows:', dup_count)
if dup_count > 0:
    df = df.drop_duplicates().reset_index(drop=True)
    print('Dropped duplicates. New shape:', df.shape)
else:
    print('No duplicates to drop')


## 6) Column types

Separate numeric and non-numeric columns for later steps.

In [ ]:
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
non_numeric_cols = df.select_dtypes(exclude=[np.number]).columns.tolist()
print('Numeric columns:', numeric_cols)
print('Non-numeric columns:', non_numeric_cols)


## 7) Imputation

Impute numeric columns with the median and categorical with the most frequent value (if needed).

In [ ]:
if df.isna().sum().sum() == 0:
    print('No missing values to impute')
else:
    num_imp = SimpleImputer(strategy='median')
    cat_imp = SimpleImputer(strategy='most_frequent')
    if numeric_cols:
        df[numeric_cols] = num_imp.fit_transform(df[numeric_cols])
    if non_numeric_cols:
        df[non_numeric_cols] = cat_imp.fit_transform(df[non_numeric_cols])
    print('Imputation complete. Remaining missing:', df.isna().sum().sum())


## 8) Outlier check (IQR method)

Count potential outliers per numeric column using IQR. This just reports counts; you can decide to remove/winsorize later.

In [ ]:
outlier_counts = {}
for c in numeric_cols:
    Q1 = df[c].quantile(0.25)
    Q3 = df[c].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    outlier_counts[c] = int(((df[c] < lower) | (df[c] > upper)).sum())

import pandas as pd
pd.Series(outlier_counts).sort_values(ascending=False)


## 9) Target handling

If 'quality' column exists we create a binary target `quality_binary` (quality >= 7 -> 1). Otherwise we'll use 'quality' or the last column as target.

In [ ]:
if 'quality' in df.columns:
    print("Found 'quality' column. Value counts:\n", df['quality'].value_counts().sort_index())
    df['quality_binary'] = (df['quality'] >= 7).astype(int)
    target = 'quality_binary'
    print("Created 'quality_binary' as target (quality >= 7 => 1).")
else:
    target = 'quality' if 'quality' in df.columns else df.columns[-1]
    print('Using target column:', target)


## 10) Encoding categorical variables

One-hot encode non-numeric columns (drop_first=True to avoid multicollinearity).

In [ ]:
if non_numeric_cols:
    df = pd.get_dummies(df, columns=non_numeric_cols, drop_first=True)
    print('One-hot encoding applied. New shape:', df.shape)
else:
    print('No categorical columns to encode')


## 11) Feature / target split

Separate X and y for modeling.

In [ ]:
X = df.drop(columns=[target])
y = df[target]
print('X shape:', X.shape)
print('y distribution:')
print(y.value_counts(normalize=True))


## 12) Train-test split

Split into train and test sets. Stratify when the target is categorical to preserve label proportions.

In [ ]:
stratify_arg = y if (y.dtype.kind in 'i' and y.nunique() <= 20) else None
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=stratify_arg
)
print('Train shape:', X_train.shape, 'Test shape:', X_test.shape)


## 13) Scaling numeric features

Fit `StandardScaler` on training numeric features and transform both train and test.

In [ ]:
numeric_after = X_train.select_dtypes(include=[np.number]).columns.tolist()
print('Numeric features to scale:', numeric_after)

scaler = StandardScaler()
X_train_num = pd.DataFrame(scaler.fit_transform(X_train[numeric_after]), columns=numeric_after, index=X_train.index)
X_test_num  = pd.DataFrame(scaler.transform(X_test[numeric_after]), columns=numeric_after, index=X_test.index)

X_train_scaled = X_train.copy()
X_test_scaled = X_test.copy()
X_train_scaled[numeric_after] = X_train_num
X_test_scaled[numeric_after] = X_test_num

X_train_scaled.head()


## 14) Save preprocessed datasets & scaler

Save train/test CSVs and the fitted scaler for later modeling.

In [ ]:
OUT_DIR = '/mnt/data/preprocessed_wine'
import os
os.makedirs(OUT_DIR, exist_ok=True)

train_df = X_train_scaled.copy()
train_df[target] = y_train

test_df = X_test_scaled.copy()
test_df[target] = y_test

train_path = os.path.join(OUT_DIR, 'wine_train_preprocessed.csv')
test_path  = os.path.join(OUT_DIR, 'wine_test_preprocessed.csv')
scaler_path = os.path.join(OUT_DIR, 'standard_scaler.joblib')

train_df.to_csv(train_path, index=False)
test_df.to_csv(test_path, index=False)
joblib.dump(scaler, scaler_path)

print('Saved files:')
print('-', train_path)
print('-', test_path)
print('-', scaler_path)


## 15) Quick summary

Final quick stats about preprocessed data.

In [ ]:
summary = {
    'original_shape': df.shape,
    'train_shape': X_train.shape,
    'test_shape': X_test.shape,
    'num_numeric_features': len(numeric_after),
    'target': target,
}
summary
